# SINDy example code

In [1]:
import numpy as np
from scipy.integrate import odeint
from sklearn.model_selection import KFold, GridSearchCV

from context import sindy
from sindy import SINDy

Generate training data

In [2]:
def rhs_harmonic_oscillator(y, t):
    dy0 = y[1]
    dy1 = -0.3 * y[0]
    return [dy0, dy1]

In [3]:
n_points = 1000

x0 = [0, 1]
t = np.linspace(0, 10, n_points)
x = odeint(rhs_harmonic_oscillator, x0, t)

train_idx = int(n_points * 0.75)
x_train, x_test = x[:train_idx], x[train_idx:]
t_train, t_test = t[:train_idx], t[train_idx:]

## Basic examples

### Simplest version

In [4]:
model = SINDy()
model.fit(x_train, t_train)

SINDy(feature_library=PolynomialFeatures(degree=2, include_bias=True, interaction_only=False),
   feature_names=None, kw={}, n_jobs=1, threshold=0.1)

In [7]:
print(model.equations())

> /home2/bdesilva/Dropbox (uwamath)/Brian/Research/SINDy/sindy/sindy/model/base.py(47)print_model()
-> components = map(term, coef, errors, input_features)
(Pdb) errors
-0.9617744686059314
(Pdb) c


TypeError: 'numpy.float64' object is not iterable